# Select subject part and subject orientation

This script limits the available controlled vocabulary terms to allowed values based on what is in the corresponding `skos:collection` for that category.

![flow chart](https://github.com/tdwg/ac/raw/master/views/code/selection_diagram.png)

## Load the data from JSON-LD files

Overall structure of JSON-LD files:

```
{
  "@context": {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "skos": "http://www.w3.org/2004/02/skos/core#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "skos:inScheme": {
      "@type": "@id"
    },
    "skos:broader": {
      "@type": "@id"
    },
    "skos:member": {
      "@type": "@id"
    }
  },
  "@graph": [
```
*array of described concepts or collections*

```
  ]
}
```


In [9]:
import requests
import json
import csv

# To load directly from GitHub, use the following base URL. To load from local files, change to empty string
github_base_url = 'https://tdwg.github.io/rs.tdwg.org/cvJson/'
#github_base_url = ''

# Function to load JSON file data into a Python data structure
def load_json_into_data_struct(path):
    if github_base_url != '':
        response_object = requests.get(github_base_url + path)
        file_text = response_object.text
    else:
        with open(github_base_url + path, 'rt', encoding='utf-8') as file_object:
            file_text = file_object.read()
    structure = json.loads(file_text)
    # uncomment the following line to view the data
    # print(json.dumps(structure, indent = 2))
    return(structure)

# Load JSON data
parts = load_json_into_data_struct('acpart.json')['@graph']
orientations = load_json_into_data_struct('acorient.json')['@graph']
parts_collections = load_json_into_data_struct('acpart_collection.json')['@graph']
orients_collections = load_json_into_data_struct('acorient_collection.json')['@graph']

## Find available organism groups from organism collections and select group

The `acpart_collection.json` file has a JSON array of organism collections:

```
[
    {
      "@id": "http://rs.tdwg.org/acviews/collections/g000",
      "@type": "http://www.w3.org/2004/02/skos/core#Collection",
      "tdwgutility:ofConcept": "http://rs.tdwg.org/acviews/values/g000",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "unspecified organism group"
        }
      ],
      "skos:member": [
```
...
```
      ]
    },
    {
      "@id": "http://rs.tdwg.org/acviews/collections/g001",
      "@type": "http://www.w3.org/2004/02/skos/core#Collection",
      "tdwgutility:ofConcept": "http://rs.tdwg.org/acviews/values/g001",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "woody angiosperms"
        }
      ],
      "skos:member": [
```
...
```
      ]
    },
    {
      "@id": "http://rs.tdwg.org/acviews/collections/g002",
      "@type": "http://www.w3.org/2004/02/skos/core#Collection",
      "tdwgutility:ofConcept": "http://rs.tdwg.org/acviews/values/g002",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "herbaceous angiosperms"
        }
      ],
      "skos:member": [
```
...
```
      ]
    },
```
...

]
```


In [5]:
# Pick a part from the collections of parts for organisms
for collectionIndex in range(len(parts_collections)):
    for label in parts_collections[collectionIndex]['skos:prefLabel']:
        if label['@language'] == 'en':
            print(collectionIndex, label['@value'])
collectionChoice = int(input('Which collection? '))
print('ID of chosen collection:', parts_collections[collectionChoice]['@id'])
partOptions = parts_collections[collectionChoice]['skos:member']

0 unspecified organism group
1 woody angiosperms
2 herbaceous angiosperms
3 gymnosperms
4 fish
5 mammals
6 birds
7 reptiles/amphibians
8 insects
Which collection? 7
ID of chosen collection: http://rs.tdwg.org/acpart/collection/g011


## Look up parts for that organism group and select part


**Example of a collection of parts for an organism group (from `acpart_collection.json`):**

```
    {
      "@id": "http://rs.tdwg.org/acpart/collection/g011",
      "@type": "http://www.w3.org/2004/02/skos/core#Collection",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "reptiles/amphibians"
        }
      ],
      "skos:definition": [
        {
          "@language": "en",
          "@value": "collection of concepts that apply to reptiles and amphibians"
        }
      ],
      "skos:member": [
        "http://rs.tdwg.org/acpart/values/p0001",
        "http://rs.tdwg.org/acpart/values/p0000",
        "http://rs.tdwg.org/acpart/values/p0013",
        "http://rs.tdwg.org/acpart/values/p0028",
        "http://rs.tdwg.org/acpart/values/p0029",
        "http://rs.tdwg.org/acpart/values/p0027",
        "http://rs.tdwg.org/acpart/values/p0026"
      ]
    },
```

**Example of controlled value and label data for a part (from `acpart.json`):**

```
    {
      "@id": "http://rs.tdwg.org/acpart/values/p0027",
      "@type": "http://www.w3.org/2004/02/skos/core#Concept",
      "rdf:value": "skull",
      "skos:inScheme": "http://rs.tdwg.org/acpart/values/p",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "skull"
        }
      ],
      "skos:definition": [
        {
          "@language": "en",
          "@value": "The part of the head consisting entirely of cranium and mandible"
        }
      ]
    },
```

In [6]:
# Pick a part from the parts in the collection for that organism
for partOptionIndex in range(len(partOptions)):
    for part in parts:
        if part['@id'] == partOptions[partOptionIndex]:
            for label in part['skos:prefLabel']:
                if label['@language'] == 'en':
                    print(partOptionIndex, label['@value'])
partChoice = int(input('Which part? '))

print()

# Look up controlled value string and IRI for part choice
for part in parts:
    if part['@id'] == partOptions[partChoice]:
        print('value for ac:subjectPartLiteral = ', part['rdf:value'])
        print('value for ac:subjectPart = ', part['@id'])


0 entire organism
1 unspecified part
2 head
3 cranium
4 mandible
5 skull
6 genitalia
Which part? 5

value for ac:subjectPartLiteral =  skull
value for ac:subjectPart =  http://rs.tdwg.org/acpart/values/p0027


## Look up available orientations for that part and select orientation

**Example of a collection of orientations for a part (from `acorient_collection.json`):**

```
    {
      "@id": "http://rs.tdwg.org/acorient/collection/p0027",
      "@type": "http://www.w3.org/2004/02/skos/core#Collection",
      "tdwgutility:ofConcept": "http://rs.tdwg.org/acpart/values/p0027",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "skull"
        }
      ],
      "skos:definition": [
        {
          "@language": "en",
          "@value": "collection of concepts appropriate for skull"
        }
      ],
      "skos:member": [
        "http://rs.tdwg.org/acorient/values/r0006",
        "http://rs.tdwg.org/acorient/values/r0007",
        "http://rs.tdwg.org/acorient/values/r0003"
      ]
    },
```


**Example of controlled value and label data for an orientation (from `subjectOrientation.json`):**

```
    {
      "@id": "http://rs.tdwg.org/acorient/values/r0006",
      "@type": "http://www.w3.org/2004/02/skos/core#Concept",
      "rdf:value": "dorsal",
      "skos:inScheme": "http://rs.tdwg.org/acorient/values/r",
      "skos:prefLabel": [
        {
          "@language": "en",
          "@value": "dorsal side"
        }
      ],
      "skos:definition": [
        {
          "@language": "en",
          "@value": "view of the dorsal side of the part"
        }
      ]
    },
```

In [10]:
# Look up orientations available for that part by checking the collection for that part
for collectionIndex in range(len(orients_collections)):
    if orients_collections[collectionIndex]['tdwgutility:ofConcept'] == partOptions[partChoice]:
        availableOrientations = orients_collections[collectionIndex]['skos:member']
        break

# Pick an orientation from in the collection for that part
for orientationChoiceIndex in range(len(availableOrientations)):
    for orientation in orientations:
        if orientation['@id'] == availableOrientations[orientationChoiceIndex]:
            for label in orientation['skos:prefLabel']:
                if label['@language'] == 'en':
                    print(orientationChoiceIndex, label['@value'])
orientationChoice = int(input('Which orientation? '))

print()

# Look up controlled value string and IRI for part choice
for orientation in orientations:
    if orientation['@id'] == availableOrientations[orientationChoice]:
        print('value for ac:subjectOrientationLiteral = ', orientation['rdf:value'])
        print('value for ac:subjectOrientation = ', orientation['@id'])


0 dorsal side
1 ventral side
2 lateral side
Which orientation? 0

value for ac:subjectOrientationLiteral =  dorsal
value for ac:subjectOrientation =  http://rs.tdwg.org/acorient/values/r0006


## End result

Selection process has resulted in the choice of valid values for either:

- controlled value strings (NOT labels) to be used with literal value properties:

```
ac:subjectPartLiteral =  "skull"
ac:subjectOrientationLiteral =  "dorsal"
```

- IRI values to be used with IRI value properties:

```
ac:subjectPart =  http://rs.tdwg.org/acviews/values/p0027
ac:subjectOrientation =  http://rs.tdwg.org/acviews/values/r0006
```

Current term proposals recommend using the IRI value properties since they have globally unique values, but in some implementations, there may be an advantage to the less opaque controlled value strings.